<h1><b><center>How to use this file with PyRat?</center></b></h1>

Google Colab provides an efficient environment for writing codes collaboratively with your group. For us, teachers, it allows to come and see your advancement from time to time, and help you solve some bugs if needed.

The PyRat software is a complex environment that takes as an input an AI file (as this file). It requires some resources as well as a few Python libraries, so we have installed it on a virtual machine for you.

PyRat is a local program, and Google Colab is a distant tool. Therefore, we need to indicate the PyRat software where to get your code! In order to submit your program to PyRat, you should follow the following steps:

1.   In this notebook, click on *Share* (top right corner of the navigator). Then, change sharing method to *Anyone with the link*, and copy the sharing link;
2.   On the machine where the PyRat software is installed, start a terminal and navigate to your PyRat directory;
3.   Run the command `python ./pyrat.py --rat "<link>" <options>`, where `<link>` is the share link copied in step 1. (put it between quotes), and `<options>` are other PyRat options you may need.

<h1><b><center>Pre-defined constants</center></b></h1>

A PyRat program should -- at each turn -- decide in which direction to move. This is done in the `turn` function later in this document, which should return one of the following constants:

In [ ]:
MOVE_DOWN = 'D'
MOVE_LEFT = 'L'
MOVE_RIGHT = 'R'
MOVE_UP = 'U'

<h1><b><center>Your coding area</center></b></h1>

Please put your functions, imports, constants, etc. between this text and the PyRat functions below. You can add as many code cells as you want, we just ask that you keep things organized (one function per cell, commented, etc.), so that it is easier for the teachers to help you debug your code!

In [ ]:
import random
import heapq
import math

In [ ]:
def create_structure():
    # Create a priority queue
    return []

def push_to_structure (structure, element) :
    # Add an element to the min-heap
    heapq.heappush(structure, element)

def pop_from_structure (structure) :
    # Extract an element from the min-heap
    return heapq.heappop(structure)

def traversal (start_vertex, graph) :
    # First we create either a LIFO or a FIFO
    queuing_structure = create_structure() 
    # Add the starting vertex with None as parent
    push_to_structure(queuing_structure,(0,start_vertex, None))
    # Initialize the outputs 
    explored_vertices = {}
    routing_table = {} 
    # Iterate while some vertices remain
    while len(queuing_structure) > 0 :
    
        # This will return the next vertex to be examined, and the choice of queuing structure will change the resulting order
        (distance_to_current_vertex,current_vertex, parent) = pop_from_structure(queuing_structure)
    
        # Tests whether the current vertex is in the list of explored vertices
        if current_vertex not in explored_vertices :
            # Mark the current_vertex as explored
            explored_vertices[current_vertex] = distance_to_current_vertex 
       
            # Update the routing table accordingly
            routing_table[current_vertex] = parent 
       
            # Examine neighbors of the current vertex
            for neighbor in list(graph[(current_vertex)].keys()): 
              # We push all unexplored neighbors to the queue
                if neighbor not in explored_vertices :              
                    distance_to_neighbor = distance_to_current_vertex + graph[current_vertex][neighbor]
                    push_to_structure(queuing_structure, (distance_to_neighbor, neighbor, current_vertex))

              
    return routing_table,explored_vertices

    

In [ ]:
def find_route (routing_table, source_location, target_location) :
  a=target_location
  r=[a]
  while a!=source_location:
    #Find parent of current vertex
    r.append(routing_table[a])
    a=routing_table[a]
  return r


In [ ]:
def move_from_locations (locations) : 
    #Convert destination into a move
    M=[]
    for i in range(len(locations)-1):
        difference = (locations[i][0] - locations[i+1][0], locations[i][1] - locations[i+1][1])  
        if difference == (0, -1) :
          M.append(MOVE_DOWN)
        elif difference == (0, 1) :
          M.append(MOVE_UP)
        elif difference == (1, 0) :
          M.append(MOVE_RIGHT)
        elif difference == (-1, 0) :
          M.append(MOVE_LEFT)
        else :
          raise Exception("Impossible move")
         
    return M

def build_meta_graph (maze_map, locations) :
    # Return the meta-graph and all necessary routing tables
    meta_graph={l : { } for l in locations}
    table={}
    for i in range(len(locations)):
      routing_table,explored_vertices=traversal(locations[i],maze_map)
      table[locations[i]]=routing_table
      for k in range(0,len(locations)):
        if k!=i:
            meta_graph[locations[i]][locations[k]]=explored_vertices[locations[k]]
    return meta_graph,table
  
def give_score (graph, current_vertex, neighbors):
      # Associate a score (length of path) to each given neighbor
      score={}
      for neighbors in neighbors:
        score[neighbors]=graph[current_vertex][neighbors]
      return score

def greedy (graph, initial_vertex, vertices_to_visit) :
    # Greedy algorithm that goes to the score minimizer until all vertices are visited
    visited_vertices=[]
    current_vertex=initial_vertex
    path=[initial_vertex]
    while vertices_to_visit!=[]:
      score=give_score(graph, current_vertex,vertices_to_visit)
      min=score[vertices_to_visit[0]]
      next=vertices_to_visit[0]
      for neighbors in vertices_to_visit:
        if score[neighbors]<min:
          min=score[neighbors]
          next=neighbors
      path.append(next)
      vertices_to_visit.remove(next)
      current_vertex=next
    return path
  
def meta_graph_route_to_route (meta_graph_route,routing_tables) :
    # Return the sequence of locations in the maze to perform a route in the meta-graph
      Move=[]
      loc=[]
      #for each cheese, we determine the route
      for i in range (len(meta_graph_route)-1):
        r=routing_tables[meta_graph_route[i]]
        c=find_route (r, meta_graph_route[i], meta_graph_route[i+1])
        c.pop()
        loc=c+loc
      loc=loc+[meta_graph_route[0]]
      #we transform the list of locations into movements
      move=move_from_locations(loc)
      return(move)  





<h1><b><center>PyRat functions</center></b></h1>

The `preprocessing` function is called at the very start of a game. It is 

---

attributed a longer time compared to `turn`, which allows you to perform 

---

intensive computations. If you store the results of these computations into **global** variables, you will be able to reuse them in the `turn` function.

*Input:*
*   `maze_map` -- **dict(pair(int, int), dict(pair(int, int), int))** -- The map of the maze where the game takes place. This structure associates each cell with the dictionry of its neighbors. In that dictionary of neighbors, keys are cell coordinates, and associated values the number of moves required to reach that neighbor. As an example, `list(maze_map[(0, 0)].keys())` returns the list of accessible cells from `(0, 0)`. Then, if for example `(0, 1)` belongs to that list, one can access the number of moves to go from `(0, 0)` to `(0, 1)` by the code `maze_map[(0, 0)][0, 1)]`.
*   `maze_width` -- **int** -- The width of the maze, in number of cells.
*   `maze_height` -- **int** -- The height of the maze, in number of cells.
*   `player_location` -- **pair(int, int)** -- The initial location of your character in the maze.
*   `opponent_location` -- **pair(int,int)** -- The initial location of your opponent's character in the maze.
*   `pieces_of_cheese` -- **list(pair(int, int))** -- The initial location of all pieces of cheese in the maze.
*   `time_allowed` -- **float** -- The time you can take for preprocessing before the game starts checking for moves.

*Output:*
*   This function does not output anything.

In [ ]:
def preprocessing (maze_map, maze_width, maze_height, player_location, opponent_location, pieces_of_cheese, time_allowed) :
    5
    loc=[player_location]+pieces_of_cheese
    meta_graph=build_meta_graph(maze_map,loc)
    score=give_score(meta_graph[0],player_location,pieces_of_cheese)
    path=greedy(meta_graph[0],player_location,pieces_of_cheese)
    global movement
    movement=meta_graph_route_to_route(path,meta_graph[1])

The `turn` function is called each time the game is waiting
for the player to make a decision (*i.e.*, to return a move among those defined above).

*Input:*
*   `maze_map` -- **dict(pair(int, int), dict(pair(int, int), int))** -- The map of the maze. It is the same as in the `preprocessing` function, just given here again for convenience.
*   `maze_width` -- **int** -- The width of the maze, in number of cells.
*   `maze_height` -- **int** -- The height of the maze, in number of cells.
*   `player_location` -- **pair(int, int)** -- The current location of your character in the maze.
*   `opponent_location` -- **pair(int,int)** -- The current location of your opponent's character in the maze.
*   `player_score` -- **float** -- Your current score.
*   `opponent_score` -- **float** -- The opponent's current score.
*   `pieces_of_cheese` -- **list(pair(int, int))** -- The location of remaining pieces of cheese in the maze.
*   `time_allowed` -- **float** -- The time you can take to return a move to apply before another time starts automatically.

*Output:*
*   A chosen move among `MOVE_UP`, `MOVE_DOWN`, `MOVE_LEFT` or `MOVE_RIGHT`.

In [ ]:
def turn (maze_map, maze_width, maze_height, player_location, opponent_location, player_score, opponent_score, pieces_of_cheese, time_allowed) :
    return movement.pop()